In [24]:
import sys
from pathlib import Path
project_path = str(Path.cwd().parent)
sys.path.append(project_path)


import pymysql
from sqlalchemy import create_engine
from sqlalchemy.orm import sessionmaker

import sys

from bs4 import BeautifulSoup
from apps.ipo.tabs import tab1, tab2, tab3, tab4, tab5
from agents import get_user_agents

from schemas.general import GeneralCreateSchema
from schemas.subscriber import SubscriberCreateSchema
from schemas.financial import FinancialCreateSchema
from schemas.shareholder import ShareholderCreateSchema
from schemas.prediction import PredictionCreateSchema

from db import models
from sqlalchemy import and_ 
import re
from datetime import datetime


import bs4
from bs4 import BeautifulSoup

from agents import get_user_agents
from utilities.time_measure import timeit


# @timeit
def scrape_company_codes(year=2021):

    current_year = datetime.now().year
    codes = []
    for year in range(year, current_year + 1):
        # '리스트가 없습니다.'라는 colspan=9 속성이 안나올때까지 순회하게 됨.
        page_data = None
        page = 1
        # while page_data is None:
        url = "http://www.ipostock.co.kr/sub03/ipo02.asp?str4=2022&str5=all"
        # url = f"http://www.ipostock.co.kr/sub03/ipo02.asp?str4={year}&str5=all&page={page}"
        # page += 1

    

        req = requests.get(url)
        soup = BeautifulSoup(req.content, "lxml", )

        page_data = soup.select_one('td[colspan="9"]')
        # if page_data is not None:
        #     break

        a_tags = soup.select('a[href*="view_pg"][class=""]')

        for tag in a_tags:
            href = tag["href"]
            code = re.search("code=(.+)&", href).group(1)
            codes.append(code)
    return codes



In [2]:
url = "http://www.ipostock.co.kr/sub03/ipo02.asp?str4=2022&str5=all"
# url = f"http://www.ipostock.co.kr/sub03/ipo02.asp?str4={year}&str5=all&page={page}"
# page += 1

from utilities import request_helper

req = requests.get(url)
soup = BeautifulSoup(req.content, "lxml", )

In [66]:
from db.dbmanager import DBManager
시작년 = 2021
마지막년 = 2024
total = 0

ipo_companies = []
delisted_companies = []

공모철회회사수 = 0

def delist(company_name):

    instance = DBManager()
    read, ci_demand_forcast_date = instance.read(company_name)

    # DB에는 공모철회가 없지만 공모철회로 데이터를 바꾸어야함.
    if read and ci_demand_forcast_date != "공모철회":
        delisted = instance.delist(company_name)


def get_name_n_code(td_name):
    a_tag = td_name.select_one('a[href*="view_pg"][class=""]')
    href = a_tag['href']
    company_code = re.search("code=(.+)&", href).group(1)
    company_name = a_tag.select_one('font').text.strip().replace('.', '')
    return company_code, company_name


def get_years(year):
    current_year = datetime.now().year
    for k in range(year, current_year + 1):
        yield k



for year in get_years(시작년):
    ipo_companies_in_year = 0
    delisted_companies_in_year = 0
    # print('*'*100)
    # print(f"year: {year}")
    page_data = None
    page = 1
    while page_data is None:
        ipo_companies_in_page = 0
        delisted_companies_in_page = 0
        url = f"http://www.ipostock.co.kr/sub03/ipo02.asp?str4={year}&str5=all&page={page}"
        # print(f"page: {page}, url: {url}")
        page += 1
        resp = requests.get(url)
        soup = BeautifulSoup(resp.content, "lxml")
        page_data = soup.select_one('td[colspan="9"]')
        if page_data is not None:
            break
        table = soup.select_one('#print > table >  tr:nth-child(4) > td > table >  tr:nth-child(3) > td > table >  tr:nth-child(4) > td > table')
        codes = []
        delisted_codes = []
        for idx, tr in enumerate(table,1):
            if isinstance(tr, bs4.element.Tag):
                td = tr.find('td', {'width': '120'})

                if td is not None:
                    td_name = tr.find('td', {'width': '*'})
                    delisting = td.text.strip()
                    company_code, company_name = get_name_n_code(td_name)

                    if delisting == "공모철회":
                        company = dict(delistted_company_name=company_name, delistted_company_code=company_code)
                        # print(company)
                        delisted_companies.append(company)
                        delisted_companies_in_page += 1

                    else:
                        company = dict(ipo_company_name=company_name, ipo_company_code=company_code)
                        # print(company)
                        ipo_companies.append(company)
                        ipo_companies_in_page += 1

ipo_companies_codes = [value for ipo_company in ipo_companies for key, value in ipo_company.items() if key == 'ipo_company_code']
delisted_companies_codes = [value for delisted_company in delisted_companies for key, value in delisted_company.items() if key == 'delistted_company_code']
return ipo_companies_codes + delisted_companies_codes
#         print('page당 공모철회 회사수',delisted_companies_in_page)

#         print('-'*50)
#         delisted_companies_in_year += delisted_companies_in_page
#     print(f'{year}연도 공모 철회 회사 수 : {delisted_companies_in_year}')
# print(f'총 공모 철회 회사 수 : {len(delisted_companies)}')
# print(delisted_companies)
#         print('page당 공모 회사수',ipo_companies_in_page)
#         print('-'*50)

#         ipo_companies_in_year += ipo_companies_in_page
#     print(f'{year}연도 공모 회사 수 : {ipo_companies_in_year}')
# print(f'총 공모 회사 수 : {len(ipo_companies)}')


                    # if delisting == '공모철회':
                    #     if td_name is not None:
                    #         a_tag = td_name.select_one('table > tr > td:nth-child(2) > a[href*="view_pg"][class=""]')
                    #         href = a_tag.get('href')
                    #         code = re.search("code=(.+)&", href).group(1)
                    #         delisted_codes.append(code)
                    #         company_name = td_name.select_one('table > tr > td:nth-child(2) > a > font').text.strip().replace('.', '')
                    #         instance = DBManager()
                    #         if isinstance(company_name, str) and company_name > 2 :
                    #             read, ci_demand_forcast_date = instance.read(company_name)
                                
                    #             # DB에는 공모철회가 없지만 공모철회로 데이터를 바꾸어야함.
                    #             if read and ci_demand_forcast_date != '공모철회':
                    #                 delisted = instance.delist(company_name)
                    #                 # 성공적으로 종목을 공모 철회로 바꾼 경우. 
                    #                 # 이미 되어 있는 경우 
                    #                 continue
                
                    #         # 회사 이름으로 쿼리 때려서 2가지로 나뉨
                    #         # 1) 반환 값이 공모철회(0) -> 
                    #         # 2) 반환 값이 공모철회(X) -> ci_demand_forecast_? 를 공모철회로 변경
                    # else:
                    #     a_tag = td_name.select_one('a[href*="view_pg"][class=""]')
                    #     href = a_tag.get('href')
                    #     code = re.search("code=(.+)&", href).group(1)
                    #     codes.append(code)
        # return codes

252 15


In [67]:
[1]+[2]

[1, 2]

In [61]:
a = [{'a':1, "b":2}, {'a':1, "b":2},{'a':1, "b":2},{'a':1, "b":2},]
result = []
for i in a:
    for value in i.values():
        result.append(value)
result        

[1, 2, 1, 2, 1, 2, 1, 2]

In [63]:
[value for i in a for value in i.values()]

[1, 2, 1, 2, 1, 2, 1, 2]